In [ ]:
pip install split_folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import splitfolders

In [ ]:
input_folder = "/content/drive/My Drive/fruits-360/Training"

output = "/content/drive/My Drive/fruits-360/output"

splitfolders.ratio(input_folder, output, seed =71, ratio = (.6, .2, .2))

Streaming output truncated to the last 5000 lines.
Copying files: 28101 files [33:59,  5.25 files/s]
Copying files: 28102 files [34:00,  5.11 files/s]
Copying files: 28103 files [34:00,  4.94 files/s]
Copying files: 28104 files [34:00,  4.85 files/s]
Copying files: 28105 files [34:00,  4.79 files/s]
Copying files: 28106 files [34:01,  4.51 files/s]
Copying files: 28107 files [34:01,  4.41 files/s]
Copying files: 28108 files [34:01,  4.64 files/s]
Copying files: 28109 files [34:01,  4.62 files/s]
Copying files: 28110 files [34:01,  4.55 files/s]
Copying files: 28111 files [34:02,  4.82 files/s]
Copying files: 28112 files [34:02,  4.54 files/s]
Copying files: 28113 files [34:02,  4.30 files/s]
Copying files: 28114 files [34:02,  4.36 files/s]
Copying files: 28115 files [34:03,  4.51 files/s]
Copying files: 28116 files [34:03,  4.26 files/s]
Copying files: 28117 files [34:03,  4.24 files/s]
Copying files: 28118 files [34:03,  4.31 files/s]
Copying files: 28119 files [34:03,  4.50 files/s]

In [ ]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
img_height, img_width = (100, 100)
batch_size = 32

train_data_dir = r"/content/drive/My Drive/fruits-360/output/train"
valid_data_dir = r"/content/drive/My Drive/fruits-360/output/val"
test_data_dir = r"/content/drive/My Drive/fruits-360/output/test"


In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.4)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical',
    subset = 'training'
)

valid_generator = train_datagen.flow_from_directory(
    valid_data_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical',
    subset = 'validation'
)




FileNotFoundError: ignored

In [ ]:
test_generator = train_datagen.flow_from_directory(
    test_data_dir,
    target_size = (img_height, img_width),
    batch_size = 1,
    class_mode = 'categorical',
    subset = 'training'
)

In [ ]:
x,y = test_generator.next()
x.shape

In [ ]:
base_model = ResNet50(include_top=False, weights = 'imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation = 'relu')(x)

predictions = Dense(train_generator.num_classes, activation = 'softmax')(x)
model = Model(inputs = base_model.input,  outputs = predictions)

for layer in base_model.layers:
  layer.trainable = False

model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.fit(train_generator,epochs = 10)


In [ ]:
model.save('/content/drive/My Drive/fruits-360/output/model100.h5')

In [ ]:
test_loss, test_acc = model.evaluate(test_generator, verbose = 2)

print('\nTest Accuracy: ', test_acc)